In [ ]:
import math

span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
E_modulus = 11000e6
k_mod = 0.8
gamma_M = 1.3

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08)]
classes = {"C24": 24, "C22": 22, "C18": 18}
f_vk = 4.0

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    w_beam = beam_weight / span
    w_total = w_surface + w_beam
    M_max = w_total * span**2 / 8
    V_max = w_total * span / 2
    I = b * h**3 / 12
    c = h / 2
    sigma_max = M_max * c / I
    Q = b * h**2 / 8
    tau_max = V_max * Q / (I * b)
    A = b * h
    tau_avg = V_max / A
    delta_max = 5 * w_total * span**4 / (384 * E_modulus * I)
    print(f"\nBeam {int(b*100)}x{int(h*100)} cm")
    print(f"Distributed load: {w_total:.3f} kN/m")
    print(f"Max moment: {M_max:.3f} kN·m")
    print(f"Max shear: {V_max:.3f} kN")
    print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
    print(f"Shear stress: {tau_max/1e6:.3f} MPa")
    print(f"Deflection: {delta_max*1000:.2f} mm")
    for cls, f_mk in classes.items():
        f_md = k_mod * f_mk / gamma_M
        f_vd = k_mod * f_vk / gamma_M
        print(f"{cls}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, OK? "
              f"Bend {'YES' if sigma_max/1e6<=f_md else 'NO'} | "
              f"Shear {'YES' if tau_max/1e6<=f_vd else 'NO'}")


In [ ]:
span = 1.95
spacing = 0.60
q_live = 3.0
q_dead = 0.15
density = 5.3
k_mod = 0.8
gamma_M = 1.3
service_deflection_ratio = 300
dynamic_factors = [1.0, 1.2, 1.5]

beam_sizes = [(0.08, 0.16), (0.06, 0.12), (0.04, 0.08), (0.02, 0.04), (0.01, 0.02)]

wood_grades = {
    "C14 (Softwood)": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    "C18 (Softwood)": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    "C22 (Softwood)": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24 (Softwood)": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    "D30 (Hardwood)": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    "D40 (Hardwood)": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
    "D50 (Hardwood)": {"f_mk": 50, "f_vk": 5.0, "E_mean": 16000e6},
}

for b, h in beam_sizes:
    beam_volume = b * h * span
    beam_weight = beam_volume * density
    w_surface = (q_live + q_dead) * spacing
    defl_ok = True
    for dyn in dynamic_factors:
        if not defl_ok: break
        w_total = (w_surface * dyn + beam_weight / span) * 1000
        M_max = w_total * span**2 / 8
        V_max = w_total * span / 2
        I = b * h**3 / 12
        c = h / 2
        sigma_max = M_max * c / I
        Q = b * h**2 / 8
        tau_max = V_max * Q / (I * b)
        A = b * h
        tau_avg = V_max / A
        print(f"\nBeam {b*100:.1f}x{h*100:.1f} cm, Dynamic factor={dyn}")
        print(f"Distributed load: {w_total/1000:.3f} kN/m")
        print(f"Max moment: {M_max/1e3:.3f} kN·m")
        print(f"Max shear: {V_max/1e3:.3f} kN")
        print(f"Bending stress: {sigma_max/1e6:.3f} MPa")
        print(f"Shear stress: {tau_max/1e6:.3f} MPa")
        for grade, props in wood_grades.items():
            f_md = k_mod * props["f_mk"] / gamma_M
            f_vd = k_mod * props["f_vk"] / gamma_M
            E = props["E_mean"]
            delta_max = 5 * w_total * span**4 / (384 * E * I)
            service_limit = span / service_deflection_ratio
            bend_ok = sigma_max/1e6 <= f_md
            shear_ok = tau_max/1e6 <= f_vd
            defl_ok = delta_max <= service_limit
            status = []
            status.append("Bend OK" if bend_ok else "BEND FAIL")
            status.append("Shear OK" if shear_ok else "SHEAR FAIL")
            status.append("Defl OK" if defl_ok else "DEFLECTION FAIL")
            print(f"{grade}: f_m,d={f_md:.2f} MPa, f_v,d={f_vd:.2f} MPa, "
                  f"Defl={delta_max:.3f} m, Status: {', '.join(status)}")


In [ ]:
import pandas as pd

# 1. Room & Floor Geometry
room_width_m = 1.87
room_length_m = 3.0

# 2. Loading Conditions
live_loads = range(3,31)
dead_load_planks_q_kPa = 0.25 # Dead load of the floor planks/decking in kN/m^2

# 3. Beam Configuration & Support
number_of_beams = 4
beam_bearing_length_m = 0.04

# 4. Material Properties & Safety Factors
k_mod = 0.8   # Modification factor for load duration and moisture (e.g., 0.8 for service class 2)
gamma_M = 1.3 # Partial safety factor for wood material properties
brick_safety_factor = 5.0 # Higher safety factor for brittle, hollow brick

# 5. Serviceability & Dynamic Limits
service_deflection_ratio = 360 # Stricter deflection limit (L/360 is common)
dynamic_factor = 1.0 # Assuming static load for simplicity. Change if floor will see dynamic use.

beam_sizes_m = [
    # (0.033, 0.07), 
    # (0.045, 0.09), 
    (0.06, 0.12), 
    (0.08, 0.16), 
    # (0.10, 0.20), 
    # (0.12, 0.24), 
]

wood_grades = {
    # "C14": {"f_mk": 14, "f_vk": 3.2, "E_mean": 7000e6},
    # "C18": {"f_mk": 18, "f_vk": 3.5, "E_mean": 9000e6},
    # "C22": {"f_mk": 22, "f_vk": 4.0, "E_mean": 10000e6},
    "C24": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
    # "D30": {"f_mk": 30, "f_vk": 4.5, "E_mean": 12000e6},
    # "D40": {"f_mk": 40, "f_vk": 5.0, "E_mean": 14000e6},
}

wood_species = {
    # "Pinus sylvestris": {"density_kN": 4.707, "durability_DC": 3.5},
    # "Picea abies": {"density_kN": 4.413, "durability_DC": 4},
    "Larix decidua": {"density_kN": 5.884, "durability_DC": 3},
    # "Pinus radiata": {"density_kN": 4.217, "durability_DC": 4.5},
}

brick_types = {
    "Spanish Hollow Brick (Assumed)": {"compressive_strength_MPa": 10.0, "is_hollow": True},
    "Solid Clay Brick (for comparison)": {"compressive_strength_MPa": 20.0, "is_hollow": False},
}
selected_brick = brick_types["Spanish Hollow Brick (Assumed)"]

results = []
span = room_width_m
beam_spacing_m = room_length_m / number_of_beams

for b, h in beam_sizes_m:
    for live_load_q_kPa in live_loads:
        for grade_name, wood_props in wood_grades.items():
            for species_name, species_props in wood_species.items():
                # Self-weight of the beam
                beam_volume = b * h * span
                beam_weight_kN = beam_volume * species_props['density_kN']
                beam_self_weight_dist_kN_m = beam_weight_kN / span
                # Surface loads (live + dead) tributary to one beam
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * beam_spacing_m
                # Total design load on the beam
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000

                # Bending and Shear Forces
                M_max_Nm = w_total_N_m * span**2 / 8
                V_max_N = w_total_N_m * span / 2 # This is also the reaction force at the support
                # Beam geometric properties
                I_m4 = b * h**3 / 12  # Moment of inertia
                A_m2 = b * h           # Cross-sectional area
                # Stresses in the beam
                sigma_max_Pa = (M_max_Nm * (h / 2)) / I_m4
                tau_max_Pa = (3 * V_max_N) / (2 * A_m2) # Max shear for rectangle
                # Allowable design stresses for the wood
                f_md_Pa = (k_mod * wood_props["f_mk"] / gamma_M) * 1e6
                f_vd_Pa = (k_mod * wood_props["f_vk"] / gamma_M) * 1e6
                # Deflection
                E_Pa = wood_props["E_mean"]
                delta_max_m = (5 * w_total_N_m * span**4) / (384 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio

                # Calculate stress on bricks
                bearing_area_m2 = b * beam_bearing_length_m
                bearing_stress_on_brick_Pa = V_max_N / bearing_area_m2
                allowable_brick_stress_Pa = (selected_brick["compressive_strength_MPa"] * 1e6) / brick_safety_factor

                # Failure mode booleans
                bend_ok = sigma_max_Pa <= f_md_Pa
                shear_ok = tau_max_Pa <= f_vd_Pa
                defl_ok = delta_max_m <= service_limit_m
                brick_ok = bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa

                results.append({
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Wood Grade": grade_name,
                    "Wood Species": species_name,
                    "Total Beam Weight": beam_weight_kN * 6,
                    "Beam Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6,
                    "Beam Shear Stress (MPa)": tau_max_Pa / 1e6,
                    "Allowable Shear (MPa)": f_vd_Pa / 1e6,
                    "Live load": live_load_q_kPa,
                    "Deflection (mm)": delta_max_m * 1000,
                    "Allowable Defl (mm)": service_limit_m * 1000,
                    "Brick Bearing Stress (MPa)": bearing_stress_on_brick_Pa / 1e6,
                    "Allowable Brick Stress (MPa)": allowable_brick_stress_Pa / 1e6,
                    "Bend OK": bend_ok,
                    "Shear OK": shear_ok,
                    "Defl OK": defl_ok,
                    "Brick OK": brick_ok
                })

df = pd.DataFrame(results)
df

In [ ]:
import pandas as pd
import numpy as np
# from scipy.integrate import cumtrapz

def cumtrapz_np(y, x):
    dx = np.diff(x)
    area = np.cumsum((y[1:] + y[:-1]) * dx / 2)
    return np.concatenate(([0.0], area))


# Geometry & layout

room_width_m = 1.87
room_length_m = 3.0

# Opening (positioned at one end along the room length)
opening_length_m = 1.3
opening_width_m = 0.7

# Joist counts (these are the counts you selected in the drawing)
number_of_common_joists = 2
number_of_tail_joists = 2

# Assumptions about layout and tributary widths:
# - The "common joist area" occupies room_length - opening_length.
# - We'll assume common joists are distributed in that length with equal center-to-center spacing.
# - Tail joists are distributed across the opening length with equal spacing leaving equal gaps to each side,
#   hence tail spacing = opening_length / (n_tail_joists + 1).
# These assumptions can be changed if your framing is different.
common_joist_area_length_m = room_length_m - opening_length_m
common_joist_spacing_m = common_joist_area_length_m / (number_of_common_joists + 1)
tail_joist_spacing_m = opening_length_m / (number_of_tail_joists + 1)

# Spans for different members
span_common_joist_m = room_width_m
span_tail_joist_m = room_width_m - opening_width_m  # short joists span from wall to header
span_header_m = opening_length_m
span_trimmer_m = room_width_m  # trimmer spans full joist direction (same as common joist span)

# point load location 'a' measured from the trimmer's left support (A)
# If the header reaction sits at a distance 'a' from the left support of the trimmer,
# define accordingly. Here we assume header reaction is located at the edge of opening:
point_load_position_a = room_width_m - opening_width_m  # as you defined (distance from left)

# -----------------------------
# Loads, materials, geometry
# -----------------------------
live_loads = range(3, 11)  # kN/m^2
dead_load_planks_q_kPa = 0.25  # kN/m^2

beam_bearing_length_m = 0.04

k_mod = 0.8
gamma_M = 1.3
brick_safety_factor = 5.0

service_deflection_ratio = 360
dynamic_factor = 1.0

beam_sizes_m = [
    (0.06, 0.12),
    (0.08, 0.16),
]

wood_grades = {
    "C24": {"f_mk": 24, "f_vk": 4.0, "E_mean": 11000e6},
}

wood_species = {
    "Larix decidua": {"density_kN_per_m3": 5.884, "durability_DC": 3},
}

brick_types = {
    "Spanish Hollow Brick (Assumed)": {"compressive_strength_MPa": 10.0, "is_hollow": True},
}
selected_brick = brick_types["Spanish Hollow Brick (Assumed)"]

# -----------------------------
# Helper: numeric max moment & deflection for a simply supported beam
# with a uniform distributed load (w N/m) and a single point load P (N) at distance a from A.
# We compute reactions, M(x), shear and numeric deflection by integrating curvature.
# -----------------------------
def analyze_beam_with_point_and_udl(L, w_N_m, P_N=0.0, a=0.0, E=1.0, I=1.0, n_points=1001):
    """
    Returns dict with reactions R_A, R_B (N), V_max (N), M_max (Nm),
    max_abs_deflection (m) computed numerically, and moment/shear arrays if needed.
    """
    # reactions for simply supported beam with UDL + single point load at a
    R_A = w_N_m * L / 2.0 + (P_N * (L - a) / L if P_N else 0.0)
    R_B = w_N_m * L / 2.0 + (P_N * a / L if P_N else 0.0)

    # x grid
    x = np.linspace(0, L, n_points)

    # bending moment diagram M(x)
    # For any x:
    # M(x) = R_A * x - w * x^2 / 2  for x <= a
    # M(x) = R_A * x - w * x^2 / 2 - P * (x - a)  for x > a
    M = R_A * x - 0.5 * w_N_m * x**2
    if P_N:
        M[x >= a] -= P_N * (x[x >= a] - a)

    # shear diagram (left shear just inside support is R_A; max shear is max(R_A, R_B) ignoring sign)
    V_left = R_A
    V_right = R_B
    V_max = max(abs(V_left), abs(V_right))

    M_max = np.max(np.abs(M))
    # curvature = M / (E*I)
    curvature = M / (E * I)

    # integrate curvature to get slope and deflection numerically
    # integrate once (slope), integrate twice (deflection) using cumulative trapezoid.
    # start slope(0)=0, deflection(0)=0 then adjust to satisfy deflection(L)=0 by removing linear trend

    # slope = np.concatenate(([0.0], cumtrapz(curvature, x)))
    # deflection = np.concatenate(([0.0], cumtrapz(slope, x)))
    slope = cumtrapz_np(curvature, x)
    deflection = cumtrapz_np(slope, x)

    # Enforce zero deflection at both ends by subtracting a linear correction:
    # compute deflection at x=0 (already 0) and x=L (defL). Subtract (defL / L) * x from deflection.
    defL = deflection[-1]
    deflection_corrected = deflection - (defL / L) * x
    max_abs_deflection = np.max(np.abs(deflection_corrected))

    return {
        "R_A": R_A,
        "R_B": R_B,
        "V_max_N": V_max,
        "M_max_Nm": M_max,
        "max_deflection_m": max_abs_deflection,
        "M_x": M,
        "x": x
    }

# -----------------------------
# Main iteration
# -----------------------------
results = []

for b, h in beam_sizes_m:
    for live_load_q_kPa in live_loads:
        for grade_name, wood_props in wood_grades.items():
            for species_name, species_props in wood_species.items():

                E_Pa = wood_props["E_mean"]
                allowable_brick_stress_Pa = (selected_brick["compressive_strength_MPa"] * 1e6) / brick_safety_factor
                f_md_Pa = (k_mod * wood_props["f_mk"] / gamma_M) * 1e6
                f_vd_Pa = (k_mod * wood_props["f_vk"] / gamma_M) * 1e6

                I_m4 = b * h**3 / 12.0
                A_m2 = b * h
                beam_self_weight_dist_kN_m = b * h * species_props['density_kN_per_m3']  # kN/m
                beam_self_weight_dist_N_m = beam_self_weight_dist_kN_m * 1000.0

                # -----------------------------------------------------------------
                # A: COMMON JOISTS (full-span joists in solid area)
                # -----------------------------------------------------------------
                beam_type = "Common Joist"
                span = span_common_joist_m
                trib_width_m = common_joist_spacing_m  # tributary width for each common joist (assumption)
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * trib_width_m
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000.0

                # simple-supported uniform load formulas
                M_max_Nm = w_total_N_m * span**2 / 8.0
                V_max_N = w_total_N_m * span / 2.0
                sigma_max_Pa = (M_max_Nm * (h / 2.0)) / I_m4
                tau_max_Pa = (3.0 * V_max_N) / (2.0 * A_m2)
                delta_max_m = (5.0 * w_total_N_m * span**4) / (384.0 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio
                bearing_stress_on_brick_Pa = V_max_N / (b * beam_bearing_length_m)

                results.append({
                    "Beam Type": beam_type,
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Live load (kN/m2)": live_load_q_kPa,
                    "Bend OK": sigma_max_Pa <= f_md_Pa,
                    "Shear OK": tau_max_Pa <= f_vd_Pa,
                    "Defl OK": delta_max_m <= service_limit_m,
                    "Brick OK": bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa,
                    "Deflection (mm)": delta_max_m * 1000.0,
                    "Allowable Defl (mm)": service_limit_m * 1000.0,
                    "Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6
                })

                # -----------------------------------------------------------------
                # B: TAIL JOISTS (short joists that land into the header)
                # -----------------------------------------------------------------
                beam_type = "Tail Joist"
                span = span_tail_joist_m
                trib_width_m = tail_joist_spacing_m  # tributary width of each tail joist
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * trib_width_m
                w_total_kN_m = (surface_load_dist_kN_m + beam_self_weight_dist_kN_m) * dynamic_factor
                w_total_N_m = w_total_kN_m * 1000.0

                M_max_Nm = w_total_N_m * span**2 / 8.0
                V_max_N = w_total_N_m * span / 2.0  # reaction on header and wall
                reaction_from_tail_joist_N = V_max_N

                I_m4 = b * h**3 / 12.0
                A_m2 = b * h
                sigma_max_Pa = (M_max_Nm * (h / 2.0)) / I_m4
                tau_max_Pa = (3.0 * V_max_N) / (2.0 * A_m2)
                delta_max_m = (5.0 * w_total_N_m * span**4) / (384.0 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio
                bearing_stress_on_brick_Pa = V_max_N / (b * beam_bearing_length_m)

                results.append({
                    "Beam Type": beam_type,
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Live load (kN/m2)": live_load_q_kPa,
                    "Bend OK": sigma_max_Pa <= f_md_Pa,
                    "Shear OK": tau_max_Pa <= f_vd_Pa,
                    "Defl OK": delta_max_m <= service_limit_m,
                    "Brick OK": bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa,
                    "Deflection (mm)": delta_max_m * 1000.0,
                    "Allowable Defl (mm)": service_limit_m * 1000.0,
                    "Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6
                })

                # -----------------------------------------------------------------
                # C: HEADER (spanning the opening, supported by trimmers)
                # -----------------------------------------------------------------
                beam_type = "Header"
                span = span_header_m

                # total load on header = sum of reactions from all tail joists (N)
                total_reaction_from_tail_N = reaction_from_tail_joist_N * number_of_tail_joists
                # model as equivalent uniform distributed load along header:
                w_from_tail_N_m = total_reaction_from_tail_N / span  # N/m

                # header also has its own self-weight
                header_self_weight_N_m = beam_self_weight_dist_N_m
                w_total_N_m = (w_from_tail_N_m + header_self_weight_N_m) * dynamic_factor

                # analyze simply-supported header under uniform w_total_N_m
                M_max_Nm = w_total_N_m * span**2 / 8.0
                V_max_N = w_total_N_m * span / 2.0
                I_m4 = b * h**3 / 12.0
                A_m2 = b * h
                sigma_max_Pa = (M_max_Nm * (h / 2.0)) / I_m4
                tau_max_Pa = (3.0 * V_max_N) / (2.0 * A_m2)
                delta_max_m = (5.0 * w_total_N_m * span**4) / (384.0 * E_Pa * I_m4)
                service_limit_m = span / service_deflection_ratio

                # point load that header transmits to each trimmer is V_max_N (reaction at trimmer)
                point_load_on_trimmer_N = V_max_N

                results.append({
                    "Beam Type": beam_type,
                    "Beam Size (mm)": f"{b*1000:.0f}x{h*1000:.0f}",
                    "Live load (kN/m2)": live_load_q_kPa,
                    "Bend OK": sigma_max_Pa <= f_md_Pa,
                    "Shear OK": tau_max_Pa <= f_vd_Pa,
                    "Defl OK": delta_max_m <= service_limit_m,
                    "Brick OK": True,  # header sits on trimmer not brick
                    "Deflection (mm)": delta_max_m * 1000.0,
                    "Allowable Defl (mm)": service_limit_m * 1000.0,
                    "Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6
                })

                # -----------------------------------------------------------------
                # D: TRIMMER (double joist) - combine its own tributary distributed load
                #    and the point load from the header (point load_on_trimmer_N at distance a).
                # -----------------------------------------------------------------
                beam_type = "Trimmer (double)"
                span = span_trimmer_m
                b_trimmer = 2.0 * b
                I_trimmer = b_trimmer * h**3 / 12.0
                A_trimmer = b_trimmer * h

                # tributary width for trimmer: it's usually half the spacing to adjacent joists.
                # We'll approximate trib width = common_joist_spacing / 2 (because trimmer shares tributary with neighbor).
                trib_width_trimmer_m = common_joist_spacing_m / 2.0
                surface_load_dist_kN_m = (live_load_q_kPa + dead_load_planks_q_kPa) * trib_width_trimmer_m
                trimmer_self_weight_dist_kN_m = b_trimmer * h * species_props['density_kN_per_m3']
                w_dist_kN_m = (surface_load_dist_kN_m + trimmer_self_weight_dist_kN_m) * dynamic_factor
                w_dist_N_m = w_dist_kN_m * 1000.0

                P_point_N = point_load_on_trimmer_N  # from header
                a = point_load_position_a

                # Use the helper to get reactions, moments, deflection numerically
                trimmer_analysis = analyze_beam_with_point_and_udl(
                    L=span,
                    w_N_m=w_dist_N_m,
                    P_N=P_point_N,
                    a=a,
                    E=E_Pa,
                    I=I_trimmer,
                    n_points=2001
                )

                # stresses & shear checks
                V_max_N = trimmer_analysis["V_max_N"]
                M_max_Nm = trimmer_analysis["M_max_Nm"]
                sigma_max_Pa = (M_max_Nm * (h / 2.0)) / I_trimmer
                tau_max_Pa = (3.0 * V_max_N) / (2.0 * A_trimmer)
                service_limit_m = span / service_deflection_ratio
                bearing_stress_on_brick_Pa = trimmer_analysis["R_A"] / (b_trimmer * beam_bearing_length_m)  # worst-case reaction on support

                results.append({
                    "Beam Type": beam_type,
                    "Beam Size (mm)": f"{b_trimmer*1000:.0f}x{h*1000:.0f}",
                    "Live load (kN/m2)": live_load_q_kPa,
                    "Bend OK": sigma_max_Pa <= f_md_Pa,
                    "Shear OK": tau_max_Pa <= f_vd_Pa,
                    "Defl OK": trimmer_analysis["max_deflection_m"] <= service_limit_m,
                    "Brick OK": bearing_stress_on_brick_Pa <= allowable_brick_stress_Pa,
                    "Deflection (mm)": trimmer_analysis["max_deflection_m"] * 1000.0,
                    "Allowable Defl (mm)": service_limit_m * 1000.0,
                    "Bending Stress (MPa)": sigma_max_Pa / 1e6,
                    "Allowable Bending (MPa)": f_md_Pa / 1e6
                })

# DataFrame
df = pd.DataFrame(results)
df
